# Tarea 1 - Taller de Deep Learning

**Fecha de entrega: 19/10/2025**  

In [18]:
import torchvision.datasets as datasets
from pathlib import Path
import torch
import torch.nn as nn
from torchvision.transforms import v2 as T
from torch.utils.data import DataLoader, random_split
import utils
import sys
import torch.optim as optim


la ki
cb879b8473582418585ac8a9fcb1e1695ebcdf97

In [19]:
import wandb




In [20]:
WANDB_TEAM_NAME = "[joacoguerrae-ort]"
WANDB_PROJECT = "[wandb-labo1-guerra-sanes]"
wandb.login()

True

In [21]:
DEVICE = "cpu"  # por defecto, usamos la CPU
if torch.cuda.is_available():
    DEVICE = "cuda"  # si hay GPU, usamos la GPU
elif torch.backends.mps.is_available():
    DEVICE = "mps"  # si no hay GPU, pero hay MPS, usamos MPS
elif torch.xpu.is_available():
    DEVICE = "xpu"  # si no hay GPU, pero hay XPU, usamos XPU

print(f"Usando {DEVICE}")

NUM_WORKERS = 0 # Win y MacOS pueden tener problemas con múltiples workers
if sys.platform == 'linux':
    NUM_WORKERS = 4  # numero de workers para cargar los datos (depende de cada caso)

print(f"Usando {NUM_WORKERS}")

Usando cuda
Usando 4


In [22]:
DATA_DIR = Path("data")



imagenette_dataset = datasets.Imagenette(
    DATA_DIR, download=False
)

transforms = T.Compose([
    T.ToImage(),
    T.Grayscale(),
    T.ToDtype(torch.float32, scale=True),
    T.Resize((224, 224))

])

imagenette_dataset_train = datasets.Imagenette(
    DATA_DIR, download=False,split = 'train', transform=transforms
)

imagenette_dataset_val = datasets.Imagenette(
    DATA_DIR, download=False,split = 'val', transform=transforms
)

name_classes = imagenette_dataset.classes
nclasses = len(name_classes)


In [23]:
imagenette_dataset_train, imagenette_dataset_test = random_split(
    imagenette_dataset_train, [0.8, 0.2]
)

In [24]:
BATCH_SIZE = 128  # tamaño del batch
def get_dataloaders(batch_size=BATCH_SIZE, num_workers=NUM_WORKERS):
    train_loader = DataLoader(
        imagenette_dataset_train,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
    )

    val_loader = DataLoader(
        imagenette_dataset_val,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
    )

    test_loader = DataLoader(
        imagenette_dataset_test,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
    )

    return train_loader, val_loader, test_loader


train_loader, val_loader, test_loader = get_dataloaders()

In [25]:
class RegularizedCNN(nn.Module):
    def __init__(self, input, n_classes, dropout_rate=0.5):
        super().__init__()
        self.conv = nn.Sequential(
            # 224*224
            nn.Conv2d(input[0], 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),  # Batch Normalization
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.1),  # Dropout espacial
            
            # 112*112
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.2),
            
            # 56*56
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.25),
            # 28*28
        )
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 28 * 28, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout_rate),
            
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout_rate),
            
            nn.Linear(256, n_classes),
        )
    
    def forward(self, x):
        x = self.conv(x)
        x = self.classifier(x)
        return x

# Crear el modelo con regularización
modelo_regularizado = RegularizedCNN(train_loader.dataset[0][0].shape, nclasses, dropout_rate=0.5).to(DEVICE)
print(modelo_regularizado)

# Optimizer con weight decay (L2 regularization)
optimizer_reg = optim.Adam(modelo_regularizado.parameters(), lr=1e-3, weight_decay=1e-4)

RegularizedCNN(
  (conv): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout2d(p=0.1, inplace=False)
    (5): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): ReLU(inplace=True)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Dropout2d(p=0.2, inplace=False)
    (10): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Dropout2d(p=0.25, inplace=False)
  )
  

In [26]:
# Configuración mejorada del sweep con más parámetros
sweep_config_improved = {
    "name": "sweep-imagenette-regularized",
    "method": "bayes",  # Bayesian optimization es más eficiente que random
    "metric": {"name": "val_loss", "goal": "minimize"},
    "parameters": {
        "learning_rate": {
            "distribution": "log_uniform_values",
            "min": 1e-5,
            "max": 1e-2
        },
        "optimizer": {
            "values": ["adam", "sgd", "adamw"]
        },
        "batch_size": {
            "values": [32, 64, 128, 256]
        },
        "dropout_rate": {
            "distribution": "uniform",
            "min": 0.2,
            "max": 0.7
        },
        "weight_decay": {
            "distribution": "log_uniform_values",
            "min": 1e-6,
            "max": 1e-3
        },
        "momentum": {
            "distribution": "uniform",
            "min": 0.85,
            "max": 0.99
        },
        "epochs": {
            "value": 30
        }
    },
    "early_terminate": {
        "type": "hyperband",
        "min_iter": 5
    }
}



In [27]:
def train_with_wandb(config=None):
    """
    Función de entrenamiento que será llamada por wandb.agent para cada configuración del sweep.
    """
    with wandb.init(config=config):
        # Obtener la configuración del sweep
        config = wandb.config
        
        # Crear dataloaders con el batch_size del sweep
        train_loader_sweep, val_loader_sweep, test_loader_sweep = get_dataloaders(
            batch_size=config.batch_size, 
            num_workers=NUM_WORKERS
        )
        
        # Crear el modelo con los parámetros del sweep
        model = RegularizedCNN(
            train_loader_sweep.dataset[0][0].shape, 
            nclasses, 
            dropout_rate=config.dropout_rate
        ).to(DEVICE)
        
        # Definir el optimizador según la configuración
        if config.optimizer == "adam":
            optimizer = optim.Adam(
                model.parameters(), 
                lr=config.learning_rate, 
                weight_decay=config.weight_decay
            )
        elif config.optimizer == "sgd":
            optimizer = optim.SGD(
                model.parameters(), 
                lr=config.learning_rate, 
                momentum=config.momentum,
                weight_decay=config.weight_decay
            )
        elif config.optimizer == "adamw":
            optimizer = optim.AdamW(
                model.parameters(), 
                lr=config.learning_rate, 
                weight_decay=config.weight_decay
            )
        
        # Criterio de pérdida
        criterion = nn.CrossEntropyLoss()
        
        # Entrenar el modelo
        train_errors, val_errors = utils.train(
            model, 
            optimizer=optimizer, 
            criterion=criterion, 
            do_early_stopping=True,
            train_loader=train_loader_sweep, 
            val_loader=val_loader_sweep, 
            device=DEVICE, 
            epochs=config.epochs,
            patience=10
        )
        
        # Evaluar en el conjunto de test al final
        model.eval()
        test_loss = 0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for inputs, targets in test_loader_sweep:
                inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                test_loss += loss.item()
                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()
        
        test_accuracy = 100. * correct / total
        test_loss = test_loss / len(test_loader_sweep)
        
        # Log de métricas finales
        wandb.log({
            "test_accuracy": test_accuracy,
            "test_loss": test_loss
        })
        
        print(f"Test Accuracy: {test_accuracy:.2f}%")

In [28]:
# Crear el sweep
sweep_id_improved = wandb.sweep(sweep_config_improved, project=WANDB_PROJECT)

print(f"Sweep ID: {sweep_id_improved}")
print(f"Para ejecutar el sweep, usa: wandb.agent('{sweep_id_improved}', function=train_with_wandb, count=20)")

Create sweep with ID: ruprmlf1
Sweep URL: https://wandb.ai/joacoguerrae-ort/%5Bwandb-labo1-guerra-sanes%5D/sweeps/ruprmlf1
Sweep ID: ruprmlf1
Para ejecutar el sweep, usa: wandb.agent('ruprmlf1', function=train_with_wandb, count=20)


In [29]:
wandb.agent('ruprmlf1', function=train_with_wandb, count=20)

wandb: Agent Starting Run: lxpu3r81 with config:
wandb: 	batch_size: 64
wandb: 	dropout_rate: 0.3364040180623441
wandb: 	epochs: 30
wandb: 	learning_rate: 1.895654925937952e-05
wandb: 	momentum: 0.9441812915072618
wandb: 	optimizer: sgd
wandb: 	weight_decay: 4.035390083182272e-05


Epoch: 001 | Train Loss: 2.31497 | Val Loss: 2.13094
Epoch: 002 | Train Loss: 2.14486 | Val Loss: 2.03087
Epoch: 003 | Train Loss: 2.07387 | Val Loss: 1.98865
Epoch: 004 | Train Loss: 1.99345 | Val Loss: 1.92981
Epoch: 005 | Train Loss: 1.94848 | Val Loss: 1.89714
Epoch: 006 | Train Loss: 1.90319 | Val Loss: 1.87722
Epoch: 007 | Train Loss: 1.88110 | Val Loss: 1.85146
Epoch: 008 | Train Loss: 1.84587 | Val Loss: 1.81214
Epoch: 009 | Train Loss: 1.82611 | Val Loss: 1.80435
Epoch: 010 | Train Loss: 1.80061 | Val Loss: 1.77857
Epoch: 011 | Train Loss: 1.77879 | Val Loss: 1.76621
Epoch: 012 | Train Loss: 1.76876 | Val Loss: 1.75522
Epoch: 013 | Train Loss: 1.74998 | Val Loss: 1.74329
Epoch: 014 | Train Loss: 1.73451 | Val Loss: 1.72145
Epoch: 015 | Train Loss: 1.71563 | Val Loss: 1.72195
Epoch: 016 | Train Loss: 1.70501 | Val Loss: 1.71015
Epoch: 017 | Train Loss: 1.69057 | Val Loss: 1.70866
Epoch: 018 | Train Loss: 1.67286 | Val Loss: 1.68675
Epoch: 019 | Train Loss: 1.64962 | Val Loss: 1

test_accuracy,▁
test_loss,▁
test_accuracy,45.1664
test_loss,1.62211


wandb: Agent Starting Run: 6xsai3nr with config:
wandb: 	batch_size: 32
wandb: 	dropout_rate: 0.2924063115158868
wandb: 	epochs: 30
wandb: 	learning_rate: 1.4554731252598313e-05
wandb: 	momentum: 0.9416585030528744
wandb: 	optimizer: adam
wandb: 	weight_decay: 1.730447508059195e-06


Epoch: 001 | Train Loss: 2.03194 | Val Loss: 1.86315
Epoch: 002 | Train Loss: 1.83643 | Val Loss: 1.77011
Epoch: 003 | Train Loss: 1.73517 | Val Loss: 1.71309
Epoch: 004 | Train Loss: 1.65914 | Val Loss: 1.66694
Epoch: 005 | Train Loss: 1.60373 | Val Loss: 1.64316
Epoch: 006 | Train Loss: 1.55292 | Val Loss: 1.60335
Epoch: 007 | Train Loss: 1.49945 | Val Loss: 1.57747
Epoch: 008 | Train Loss: 1.44319 | Val Loss: 1.58158
Epoch: 009 | Train Loss: 1.39027 | Val Loss: 1.52805
Epoch: 010 | Train Loss: 1.33288 | Val Loss: 1.50865
Epoch: 011 | Train Loss: 1.27876 | Val Loss: 1.49514
Epoch: 012 | Train Loss: 1.21456 | Val Loss: 1.47717
Epoch: 013 | Train Loss: 1.14955 | Val Loss: 1.45166
Epoch: 014 | Train Loss: 1.09358 | Val Loss: 1.43346
Epoch: 015 | Train Loss: 1.03017 | Val Loss: 1.43935
Epoch: 016 | Train Loss: 0.98483 | Val Loss: 1.43629
Epoch: 017 | Train Loss: 0.92003 | Val Loss: 1.41972
Epoch: 018 | Train Loss: 0.85659 | Val Loss: 1.40816
Epoch: 019 | Train Loss: 0.79634 | Val Loss: 1

...

Traceback (most recent call last):
  File "/home/rami/.conda/envs/Taller_DL/lib/python3.11/site-packages/wandb/agents/pyagent.py", line 297, in _run_job
    self._function()
  File "/tmp/ipykernel_35337/2715244269.py", line 20, in train_with_wandb
    ).to(DEVICE)
      ^^^^^^^^^^
  File "/home/rami/.conda/envs/Taller_DL/lib/python3.11/site-packages/torch/nn/modules/module.py", line 1340, in to
    return self._apply(convert)
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/rami/.conda/envs/Taller_DL/lib/python3.11/site-packages/torch/nn/modules/module.py", line 900, in _apply
    module._apply(fn)
  File "/home/rami/.conda/envs/Taller_DL/lib/python3.11/site-packages/torch/nn/modules/module.py", line 900, in _apply
    module._apply(fn)
  File "/home/rami/.conda/envs/Taller_DL/lib/python3.11/site-packages/torch/nn/modules/module.py", line 927, in _apply
    param_applied = fn(param)
                    ^^^^^^^^^
  File "/home/rami/.conda/envs/Taller_DL/lib/python3.11/site-packages/torch/n

Traceback (most recent call last):
  File "/tmp/ipykernel_35337/2715244269.py", line 20, in train_with_wandb
    ).to(DEVICE)
      ^^^^^^^^^^
  File "/home/rami/.conda/envs/Taller_DL/lib/python3.11/site-packages/torch/nn/modules/module.py", line 1340, in to
    return self._apply(convert)
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/rami/.conda/envs/Taller_DL/lib/python3.11/site-packages/torch/nn/modules/module.py", line 900, in _apply
    module._apply(fn)
  File "/home/rami/.conda/envs/Taller_DL/lib/python3.11/site-packages/torch/nn/modules/module.py", line 900, in _apply
    module._apply(fn)
  File "/home/rami/.conda/envs/Taller_DL/lib/python3.11/site-packages/torch/nn/modules/module.py", line 927, in _apply
    param_applied = fn(param)
                    ^^^^^^^^^
  File "/home/rami/.conda/envs/Taller_DL/lib/python3.11/site-packages/torch/nn/modules/module.py", line 1326, in convert
    return t.to(
           ^^^^^
torch.OutOfMemoryError: CUDA out of memory. Tried to allocate

Traceback (most recent call last):
  File "/home/rami/.conda/envs/Taller_DL/lib/python3.11/site-packages/wandb/agents/pyagent.py", line 297, in _run_job
    self._function()
  File "/tmp/ipykernel_35337/2715244269.py", line 20, in train_with_wandb
    ).to(DEVICE)
      ^^^^^^^^^^
  File "/home/rami/.conda/envs/Taller_DL/lib/python3.11/site-packages/torch/nn/modules/module.py", line 1340, in to
    return self._apply(convert)
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/rami/.conda/envs/Taller_DL/lib/python3.11/site-packages/torch/nn/modules/module.py", line 900, in _apply
    module._apply(fn)
  File "/home/rami/.conda/envs/Taller_DL/lib/python3.11/site-packages/torch/nn/modules/module.py", line 900, in _apply
    module._apply(fn)
  File "/home/rami/.conda/envs/Taller_DL/lib/python3.11/site-packages/torch/nn/modules/module.py", line 927, in _apply
    param_applied = fn(param)
                    ^^^^^^^^^
  File "/home/rami/.conda/envs/Taller_DL/lib/python3.11/site-packages/torch/n

Traceback (most recent call last):
  File "/tmp/ipykernel_35337/2715244269.py", line 20, in train_with_wandb
    ).to(DEVICE)
      ^^^^^^^^^^
  File "/home/rami/.conda/envs/Taller_DL/lib/python3.11/site-packages/torch/nn/modules/module.py", line 1340, in to
    return self._apply(convert)
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/rami/.conda/envs/Taller_DL/lib/python3.11/site-packages/torch/nn/modules/module.py", line 900, in _apply
    module._apply(fn)
  File "/home/rami/.conda/envs/Taller_DL/lib/python3.11/site-packages/torch/nn/modules/module.py", line 900, in _apply
    module._apply(fn)
  File "/home/rami/.conda/envs/Taller_DL/lib/python3.11/site-packages/torch/nn/modules/module.py", line 927, in _apply
    param_applied = fn(param)
                    ^^^^^^^^^
  File "/home/rami/.conda/envs/Taller_DL/lib/python3.11/site-packages/torch/nn/modules/module.py", line 1326, in convert
    return t.to(
           ^^^^^
torch.OutOfMemoryError: CUDA out of memory. Tried to allocate

Traceback (most recent call last):
  File "/home/rami/.conda/envs/Taller_DL/lib/python3.11/site-packages/wandb/agents/pyagent.py", line 297, in _run_job
    self._function()
  File "/tmp/ipykernel_35337/2715244269.py", line 20, in train_with_wandb
    ).to(DEVICE)
      ^^^^^^^^^^
  File "/home/rami/.conda/envs/Taller_DL/lib/python3.11/site-packages/torch/nn/modules/module.py", line 1340, in to
    return self._apply(convert)
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/rami/.conda/envs/Taller_DL/lib/python3.11/site-packages/torch/nn/modules/module.py", line 900, in _apply
    module._apply(fn)
  File "/home/rami/.conda/envs/Taller_DL/lib/python3.11/site-packages/torch/nn/modules/module.py", line 900, in _apply
    module._apply(fn)
  File "/home/rami/.conda/envs/Taller_DL/lib/python3.11/site-packages/torch/nn/modules/module.py", line 927, in _apply
    param_applied = fn(param)
                    ^^^^^^^^^
  File "/home/rami/.conda/envs/Taller_DL/lib/python3.11/site-packages/torch/n